In [1]:
import caffe
import numpy as np

In [2]:
fcn32s_proto_3channels = '/data/soft/digits/digits/jobs/20170225-224205-aa61/deploy.prototxt'

In [3]:
fcn32s_HHAmodel = '/data/soft/digits/digits/jobs/20170223-102514-9981/snapshot_iter_105700.caffemodel'
fcn32s_cubNmodel = '/data/soft/digits/digits/jobs/20170224-152658-14f6/snapshot_iter_105700.caffemodel'
fcn32s_jetNmodel = '/data/soft/digits/digits/jobs/20170223-201441-5d22/snapshot_iter_105700.caffemodel'
fcn32s_RGB100model = '/data/soft/digits/digits/jobs/20170225-224205-aa61/snapshot_iter_105700.caffemodel'

In [4]:
mono_deploy = '/home/jguerry/workspace/dl_data_sunrgbd/MODELS/fcn32s_optim/deploy.prototxt'
fuse_deploy = '/home/jguerry/workspace/dl_data_sunrgbd/MODELS/fcn32s_optim_fuse/deploy_fuse_concat012.prototxt'

weights_path_0=fcn32s_RGB100model
weights_path_1=fcn32s_HHAmodel


weights_path_fuse='/home/jguerry/workspace/dl_data_sunrgbd/MODELS/fcn32s_optim_fuse/fuse_concat012_RGB-HHA.caffemodel'

In [5]:
def transfer(new_net, net, prefix='', suffix=''):
    # from fcn.berkeleyvision.org
    for p in net.params:
        #print "Working on", p, ':'
        p_new = prefix + p + suffix
        if p_new not in new_net.params:
            print 'dropping', p
            continue
        for i in range(len(net.params[p])):
            if i > (len(new_net.params[p_new]) - 1):
                print 'dropping', p, i
                break
            if net.params[p][i].data.shape != new_net.params[p_new][i].data.shape:
                print 'coercing', p, i, 'from', net.params[p][i].data.shape, 'to', new_net.params[p_new][i].data.shape
            else:
                print 'copying', p, ' -> ', p_new, i
            new_net.params[p_new][i].data.flat = net.params[p][i].data.flat

In [6]:
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [7]:
monoNet = caffe.Net(mono_deploy, caffe.TEST)
# monoBlobs = [(k, v.data.shape) for k, v in monoNet.blobs.items()]
# monoLayers = [(k, v[0].data.shape, v[1].data.shape) for k, v in monoNet.params.items() if k != 'upscore37']

In [8]:
fuseNet = caffe.Net(fuse_deploy, caffe.TEST)
# fuseBlobs = [(k, v.data.shape) for k, v in fuseNet.blobs.items()]
# fuseLayers = [(k, v[0].data.shape, v[1].data.shape) for k, v in fuseNet.params.items() if k not in ['upscore37_data0','upscore37_data1','upscore_sum', 'upscore_concat','upscore_rc_corrected']]

In [9]:
monoNet.copy_from(weights_path_0,)
transfer(fuseNet,monoNet,prefix = 'data0_')
monoNet.copy_from(weights_path_1)
transfer(fuseNet,monoNet,prefix = 'data1_')

copying conv1_1  ->  data0_conv1_1 0
copying conv1_1  ->  data0_conv1_1 1
copying conv1_2  ->  data0_conv1_2 0
copying conv1_2  ->  data0_conv1_2 1
copying conv2_1  ->  data0_conv2_1 0
copying conv2_1  ->  data0_conv2_1 1
copying conv2_2  ->  data0_conv2_2 0
copying conv2_2  ->  data0_conv2_2 1
copying conv3_1  ->  data0_conv3_1 0
copying conv3_1  ->  data0_conv3_1 1
copying conv3_2  ->  data0_conv3_2 0
copying conv3_2  ->  data0_conv3_2 1
copying conv3_3  ->  data0_conv3_3 0
copying conv3_3  ->  data0_conv3_3 1
copying conv4_1  ->  data0_conv4_1 0
copying conv4_1  ->  data0_conv4_1 1
copying conv4_2  ->  data0_conv4_2 0
copying conv4_2  ->  data0_conv4_2 1
copying conv4_3  ->  data0_conv4_3 0
copying conv4_3  ->  data0_conv4_3 1
copying conv5_1  ->  data0_conv5_1 0
copying conv5_1  ->  data0_conv5_1 1
copying conv5_2  ->  data0_conv5_2 0
copying conv5_2  ->  data0_conv5_2 1
copying conv5_3  ->  data0_conv5_3 0
copying conv5_3  ->  data0_conv5_3 1
copying fc6  ->  data0_fc6 0
copying f

In [11]:
for k, v in fuseNet.params.items():
    if 'upscore' in k:
        for i in range(len(fuseNet.params[k])):
            fuseNet.params[k][i].data.flat = monoNet.params['upscore37'][i].data.flat
            print(k,i)
            print(np.sum(fuseNet.params[k][i].data)) #flat = net.params[p][i].data.flat

('data0_upscore37', 0)
37888.0
('data1_upscore37', 0)
37888.0
('fuse_concat0_upscore37', 0)
37888.0
('fuse_concat1_upscore37', 0)
37888.0
('fuse_concat2_upscore37', 0)
37888.0


In [14]:
#### SAVING
print "Saving output model to %s " % weights_path_fuse
fuseNet.save(weights_path_fuse)

Saving output model to /home/jguerry/workspace/dl_data_sunrgbd/MODELS/fcn32s_optim_fuse/fuse_concat012_RGB-HHA.caffemodel 


In [15]:
del fuseNet
del monoNet